# Data Science Notebook 

In [1]:
import os, sys
import warnings
import pandas as pd
import numpy as np
from IPython.display import display, HTML

In [2]:
# Turn off jedi
%config IPCompleter.use_jedi=False

workdir = os.path.dirname(os.getcwd())

sys.path.append(workdir)

pd.options.display.max_columns = None

warnings.filterwarnings("ignore")

display(
    HTML(
        """
        <style>
        .container { width:100% !important; }
        </style>
        """
    )
)

In [ ]:
class VersionController:

    def __init__(self, init_version='0.0.0', sep = '.', sufix=''):
        self.version = init_version

In [3]:
df = pd.read_parquet("../datasets/raw/data_raw_mg+pocos-de-caldas_venda_imoveis.parquet")

In [4]:
df

,search_id,search_date,seacrh_action,search_type,search_localization,search_page,displayAddressType,amenities,usableAreas,constructionStatus,listingType,description,title,stamps,createdAt,floors,unitTypes,nonActivationReason,providerId,propertyType,unitSubTypes,unitsOnTheFloor,legacyId,id,portal,unitFloor,parkingSpaces,updatedAt,address_country,address_zipCode,address_geoJson,address_city,address_streetNumber,address_level,address_precision,address_confidence,address_stateAcronym,address_source,point_lon,point_source,point_lat,address_ibgeCityId,address_zone,address_street,address_locationId,address_district,address_name,address_state,address_neighborhood,address_poisList,address_complement,address_pois,address_valuableZones,valuableZones_city,valuableZones_zone,valuableZones_name,valuableZones_id,valuableZones_state,valuableZones_category,suites,publicationType,externalId,bathrooms,usageTypes,totalAreas,whatsappNumber,bedrooms,acceptExchange,pricingInfos_yearlyIptu,pricingInfos_price,pricingInfos_businessType,pricingInfos_monthlyCondoFee,showPrice,resale,buildings,capacityLimit,status,hasAddress,isDevelopment,isInactive,isDefaulterInactive,pricingInfos,pricingInfo_monthlyCondoFee,pricingInfo_period,pricingInfo_price,pricingInfo_rentalPrice,pricingInfo_rentalTotalPrice,pricingInfo_salePrice,pricingInfo_showPrice,pricingInfo_yearlyIptu,pricingInfo_priceVariation,pricingInfo_warranties,pricingInfo_businessType,pricingInfo_businessLabel,pricingInfo_businessDescription,pricingInfo_isSale,pricingInfo_isRent,subtitle,businessTypeContext,preview,showPhoneButton,link,isSpecialRent,rentalInfo_period,rentalInfo_warranties,sourceId,isAllowedConcierge,pricingInfo_rawPrice,SALE_isSale,SALE_price,SALE_rawPrice,SALE_salePrice,SALE_businessType,SALE_yearlyIptu,SALE_monthlyCondoFee,showRentalDatazapStamp,listingsCount,pricingInfo_rawPeriod,RENTAL_isRent,RENTAL_price,RENTAL_rawPrice,RENTAL_period,RENTAL_rawPeriod,RENTAL_warranties,RENTAL_rentalPrice,RENTAL_businessType,RENTAL_rentalTotalPrice,RENTAL_yearlyIptu,RENTAL_monthlyCondoFee
0,2583449784__2023_01_07_09_55_56,2023-01-07T09:55:56.268959,venda,imoveis,mg+pocos-de-caldas,1.0,NEIGHBORHOOD,,200,ConstructionStatus_NONE,USED,Casa antiga com potencial para adaptação comer...,POÇOS DE CALDAS - Casa Padrão - CENTRO,None,2022-10-05T07:57:32.226Z,nan,HOME,NonActivationReason_NONE,64452,UNIT,None,0.0,,2583449784,GRUPOZAP,0.0,1,2022-12-10T00:37:22.318Z,BR,37701024,,Poços de Caldas,,STREET,GEOMETRIC_CENTER,VALID_STREET,MG,CORREIOS,nan,nan,nan,,Bairros,,BR>Minas Gerais>NULL>Pocos de Caldas>Barrios>C...,,,Minas Gerais,Centro,BR:Mirian’s|BR:Pesto Cozinha Bar|BR:Tenda do H...,nan,None,None,nan,nan,nan,nan,nan,nan,,SUPER_PREMIUM,CA00896,3,RESIDENTIAL,200,,6,0.0,nan,650000,SALE,nan,1.0,0.0,0.0,nan,ACTIVE,1.0,0.0,0.0,0.0,nan,,nan,R$ 650.000,nan,nan,R$ 650.000,1.0,,None,nan,SALE,Comprar,para comprar,1.0,0.0,Casa para comprar em,SALE,0.0,1.0,/imovel/venda-casa-6-quartos-centro-pocos-de-c...,nan,nan,nan,b4a14cd1-9bce-372a-960a-3114330f160d,False,650000,True,R$ 650.000,650000,R$ 650.000,SALE,,,False,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,2599668745__2023_01_07_09_55_56,2023-01-07T09:55:56.269306,venda,imoveis,mg+pocos-de-caldas,1.0,ALL,SAUNA| POOL| ELEVATOR| BARBECUE_GRILL| PETS_AL...,112,ConstructionStatus_NONE,USED,Lindo Apartamento no coração do centro em Poço...,Uma vista de arrancar suspiros!!!,None,2022-12-15T15:51:36.165Z,8.0,APARTMENT,NonActivationReason_NONE,,UNIT,None,2.0,,2599668745,GRUPOZAP,8.0,1,2022-12-15T15:59:06.674Z,BR,37701011,,Poços de Caldas,966,STREET,RANGE_INTERPOLATED,VALID_STREET,MG,CORREIOS,-46.561767,GOOGLE,-21.788204,,Bairros,Rua Rio de Janeiro,BR>Minas Gerais>NULL>Pocos de Caldas>Barrios>C...,,,Minas Gerais,Centro,CH:DentalStudio Odontologia Especializada|SC:S...,nan,None,None,nan,nan,nan,nan,nan,nan,1,SUPER_PREMIUM,19K3L67,2,RESIDENTIAL,112,35984374645,3,0.0,1600,870000,SALE,650,1.0,0.0,1.0,nan,ACTIVE,1.0,0.0,0.0,0.0,nan,R$ 650,nan,R$ 870.000,nan,nan,R$ 870.000,1.0,R$ 1

In [ ]:
from pc_zap_scrapper.scrap import search_estates
from pc_zap_scrapper import ACTION, LOCALIZATION, TYPE

search_estates(ACTION, TYPE, LOCALIZATION)